In [1]:
import torch
from torch import nn
import numpy as np
import os
from torch.utils.data import Dataset, IterableDataset, DataLoader
import math
#from torchviz import make_dot

ModuleNotFoundError: No module named 'torchviz'

In [95]:
a = torch.tensor([1.], requires_grad=True)
b = a.detach().requires_grad_()
c = b * 2
c.backward()

In [99]:
b.data = torch.tensor(0)

In [135]:
h = torch.tensor(1., requires_grad=True)
s = torch.tensor(0., requires_grad=True)
# s = 0
c = torch.tensor(2., requires_grad=True)
for i in range(10):
    h = h.detach() * c
#     h.retain_grad()
    s = s + h
s.backward()

In [137]:
h = torch.tensor(1., requires_grad=True)
c = h.detach()
d = h.detach().clone()

In [147]:
h.data = torch.tensor(0)

In [153]:
a = torch.arange(5., requires_grad=True)
b = a**2
c = a.detach().clone() # error with detach(), wrong result with .data
c.zero_()
b.sum().backward()
print(a.grad)

tensor([0., 2., 4., 6., 8.])


In [33]:
def gen_a_copy_sample(T):
    # X: {0,...,7} x 10 + {8} x T + {9} x 1 + {8} x 10
    # Y: {8} x (T+10) + {0...7} x 10 
    x = torch.cat([torch.randint(0, 8, size=[10],dtype=torch.int), \
            torch.ones(10-1,dtype=torch.int)*8, \
            torch.tensor([9]), \
            torch.ones(10, dtype=torch.int)*8])
    y = torch.ones(T+20,dtype=torch.int)*8
    y[-10:] = x[:10]
    return x, y

In [38]:
class CopyDataset(IterableDataset):
    def __init__(self, T, n_samples, device='cpu'):
        super(CopyDataset).__init__()
        self.T = T
        self.n_samples = n_samples
        self.device = device
    def __iter__(self):
        for i in range(self.n_samples):
            yield gen_a_copy_sample(self.T)

In [44]:
train_set = CopyDataset(10, 10)
train_loader = DataLoader(train_set, batch_size=5)

[tensor([[3, 7, 3, 4, 5, 3, 7, 4, 6, 6, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 8, 8, 8, 8,
         8, 8, 8, 8, 8, 8],
        [4, 6, 2, 0, 2, 2, 0, 6, 3, 4, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 8, 8, 8, 8,
         8, 8, 8, 8, 8, 8],
        [1, 4, 1, 5, 4, 6, 3, 4, 1, 5, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 8, 8, 8, 8,
         8, 8, 8, 8, 8, 8],
        [0, 7, 7, 1, 7, 2, 2, 3, 4, 4, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 8, 8, 8, 8,
         8, 8, 8, 8, 8, 8],
        [4, 1, 7, 0, 0, 0, 5, 4, 7, 5, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 8, 8, 8, 8,
         8, 8, 8, 8, 8, 8]]), tensor([[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 3, 7, 3, 4,
         5, 3, 7, 4, 6, 6],
        [8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 4, 6, 2, 0,
         2, 2, 0, 6, 3, 4],
        [8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 1, 4, 1, 5,
         4, 6, 3, 4, 1, 5],
        [8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 0, 7, 7, 1,
         7, 2, 2, 3, 4, 4],
        [8, 8, 8

In [ ]:
class DoubleCharValDataset(Dataset):
    def __init__(self, vocab_size, seq_len, copy_num, n_samples, device):
        super(DoubleCharValDataset).__init__()
        self.n_samples = n_samples
        self.X, self.Y = make_data_points(vocab_size, seq_len, copy_num, n_samples, device)
    def __len__(self):
        return self.n_samples
    def __getitem__(self, idx):
        return self.X[idx,:], self.Y[idx,:]

In [55]:
def gen_n_copy_samples(T, n):
    
    x = torch.cat([torch.randint(0, 8, size=[n, 10],dtype=torch.int), \
        torch.ones([n,T-1], dtype=torch.int)*8, \
        torch.ones([n,1],dtype=torch.int)*9, \
        torch.ones([n,10], dtype=torch.int)*8], dim=1)
    
    y = torch.ones([n,T+20],dtype=torch.int)*8
    y[:,-10:] = x[:,:10]
    
    return x, y

In [61]:
class CopyDatasetValidation(Dataset):
    def __init__(self, T, n_samples):
        super(CopyDatasetValidation).__init__()
        self.T = T 
        self.n_samples = n_samples
        self.X, self.Y = gen_n_copy_samples(self.T, self.n_samples)
    def __len__(self):
        return self.n_samples
    def __getitem__(self, idx):
        return self.X[idx,:], self.Y[idx,:]

In [62]:
val_set = CopyDatasetValidation(5, 2)

In [65]:
for i in range(2):
    print(val_set[i])

(tensor([5, 0, 3, 0, 1, 3, 7, 1, 0, 4, 8, 8, 8, 8, 9, 8, 8, 8, 8, 8, 8, 8, 8, 8,
        8], dtype=torch.int32), tensor([8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 5, 0, 3, 0, 1, 3, 7, 1, 0,
        4], dtype=torch.int32))
(tensor([5, 4, 1, 1, 2, 7, 1, 6, 1, 4, 8, 8, 8, 8, 9, 8, 8, 8, 8, 8, 8, 8, 8, 8,
        8], dtype=torch.int32), tensor([8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 5, 4, 1, 1, 2, 7, 1, 6, 1,
        4], dtype=torch.int32))


In [50]:
torch.ones([2,20-1], dtype=torch.int)*8

tensor([[8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8],
        [8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]],
       dtype=torch.int32)

In [51]:
torch.ones([2,1],dtype=torch.int)*9

tensor([[9],
        [9]], dtype=torch.int32)

In [52]:
torch.ones([2,10], dtype=torch.int)*8

tensor([[8, 8, 8, 8, 8, 8, 8, 8, 8, 8],
        [8, 8, 8, 8, 8, 8, 8, 8, 8, 8]], dtype=torch.int32)

In [79]:
def tile(a, dim, n_tile, device):
    init_dim = a.size(dim)
    repeat_idx = [1] * a.dim()
    repeat_idx[dim] = n_tile
    a = a.repeat(*(repeat_idx))
    order_index = torch.cuda.LongTensor(np.concatenate([init_dim * np.arange(n_tile) + i for i in range(init_dim)]))
    return torch.index_select(a, dim, order_index)

def make_data_points(vocab_size, seq_len, copy_num, n_samples, device):
    # return 
    num_fill = math.ceil(seq_len/copy_num)
    X1 = torch.randint(0, vocab_size, size=[n_samples, num_fill], device=device)
    X2 = torch.ones(size=[n_samples, seq_len - num_fill], device=device)*(vocab_size + 1)
    X = torch.cat([X1, X2], dim=1)
    Y = tile(X, dim=1, n_tile=copy_num, device=device)[:,:seq_len]
    return X.squeeze_(), Y.squeeze_()

In [80]:
make_data_points(4, 10, 2, 2, 'cuda:0')

(tensor([[2., 1., 3., 1., 0., 5., 5., 5., 5., 5.],
         [3., 3., 2., 1., 1., 5., 5., 5., 5., 5.]], device='cuda:0'),
 tensor([[2., 2., 1., 1., 3., 3., 1., 1., 0., 0.],
         [3., 3., 3., 3., 2., 2., 1., 1., 1., 1.]], device='cuda:0'))

In [73]:
11//2

5

In [5]:
import math
from typing import List, Union
!pip3 install Cython
import numpy as np
import pytorch_lightning as pl
import torch
from torch._C import dtype
# from torch.utils.data import dataloader
# from torch.utils.data.dataloader import DataLoader
from torch.utils.data import Dataset, IterableDataset, DataLoader

In [3]:
def gen_n_copy_samples(delay: int, n: int, rand):
    rng = np.random.default_rng(rand)
    # torch.randint(0, 8, size=[n, 10], dtype=torch.long)
    rand_seq_np = rng.integers(0, 8, size=[n, 10], dtype=np.int64)
    x = torch.cat([torch.from_numpy(rand_seq_np),
                   torch.ones([n, delay-1], dtype=torch.long)*8,
                   torch.ones([n, 1], dtype=torch.long)*9,
                   torch.ones([n, 10], dtype=torch.long)*8], dim=1)
    y = torch.ones([n, delay+20], dtype=torch.long)*8
    y[:, -10:] = x[:, :10]
    torch.save(x, f'data/copymem_{delay}_{rand}_10_x')
    torch.save(y, f'data/copymem_{delay}_{rand}_10_y')

In [8]:
gen_n_copy_samples(3,2000000,1)

In [9]:
gen_n_copy_samples(5,2000000,1)

In [10]:
gen_n_copy_samples(10,2000000,1)

In [13]:
x = torch.load('data/copymem_3_1_10_x')

In [14]:
x.shape

torch.Size([2000000, 23])

In [15]:
x1 = torch.load('data/copymem_5_1_10_x')

In [16]:
x1.shape

torch.Size([2000000, 25])

In [17]:
T = 10
delay = 5

In [18]:
dims = (T*2+delay)

In [22]:
print(dims)

25


In [ ]:
y = torch.load('data/copymem_3_1_10_y')

In [4]:
gen_n_copy_samples(15,2000000,1)